In [ ]:
import os
import cudf
import numpy as np
import nvtabular as nvt
from cudf.io.parquet import ParquetWriter

data_path = '/datasets/criteo/crit_orig/'

train_set = [os.path.join(data_path, x) for x in os.listdir(data_path) if x.startswith("day_") and not x.endswith("parquet")] 
train_set.sort()

cont_names = ["I" + str(x) for x in range(1, 14)]
cat_names = ["C" + str(x) for x in range(1, 27)]
cols = ["label"] + cont_names + cat_names


dtypes = {}
dtypes["label"] = np.int64
for x in cont_names:
    dtypes[x] = np.int64
for x in cat_names:
    dtypes[x] = "hex"
    
    
def file_to_pq(target_files, file_type, output_folder='par_out', cols=[], dtypes=None):
    target_files.sort()
    tar_data = nvt.Dataset(target_files, engine=file_type, names=cols, part_mem_fraction=0.05, sep='\t', dtypes=dtypes)
    count = 0
    old_file_path = None
    writer = None
    tar = tar_data.to_iter()
    for gdf in tar:
        file_path = os.path.join(output_folder, os.path.split(tar.cur_path)[1].split('.')[0])
        path = f'{file_path}.parquet'
        if file_path != old_file_path:
            writer = ParquetWriter(path)
        writer.write_table(gdf)
        old_file_path = file_path
        del gdf
path_out = '/raid/criteo/tests/demo_out/'
file_to_pq(train_set, 'csv', output_folder=path_out, cols=cols, dtypes=dtypes)